# Image processing pipeline

In [48]:
# Import necessary packages
%pip install imutils opencv-python simplekml palettable gdown
import gdown
import os
import glob
import rasterio as ro
import re
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj
from tqdm import tqdm
from scipy.interpolate import griddata
from PIL import Image
from skimage.measure import block_reduce
import cv2
import imageio
import imutils
import helper_functions as hf
from palettable import colorbrewer
from scipy import ndimage
cv2.ocl.setUseOpenCL(False)

Note: you may need to restart the kernel to use updated packages.


In [52]:
# Variables
gdrive = False # Download data from google drive
link = 'https://drive.google.com/file/d/1ANJ_Hq3C42Zfka_wIDX7wgbH_hwcUbGn/view?usp=sharing' # gdrive link
imgDir = 'data/120' # Local directory to images
imageType, cmap = False, colorbrewer.get_map('RdYlBu', 'diverging', 11, reverse=True).mpl_colormap # True if RGB images, False if numpy arrays. Choose matplotlib cmap for thermal
pathFile = 'data/lsm/Flight 1.csv' # Flight path .csv file
utcDiff = -1 # Time difference to UTC (-1 for BST)
pathColumns = ['timestamp','latitude','longitude','altitude(m)','pitch(deg)','roll(deg)','yaw(deg)'] # Columns used from the path file
minAlt, maxAlt, altCol = 119.7, 120.3, 'altitude(m)' # Altitude criteria for using imagery, if wanted else False
imgTimes = False#[1619811011923,1619811099149] # Specify times if wanted else False
utmZone, hemisphere = '31n', 'north' # Specify UTM coordinate zone
pxSize = 0.27 # Specify pixel size (m) - for now
resolution = 6 # Other resolution for plots
useCentre = True # True if only using center of images
skip = 20 # Plot every nth image for path

In [50]:
# Download from google drive if gdrive is True
#%capture
if gdrive: 
    zippath = imgDir+'.zip'
    downLink = link.split('/d/')[1].split('/view')[0]
    ! gdown -O $zippath 'https://drive.google.com/uc?export=download&id='$downLink
    ! unzip $zippath -d $imgDir
    ! rm $zippath

In [ ]:
## Get image dataframe with corresponding properties extracted frpm path file
fileTypes = ('.jpg','.png','.tif') if imageType else ('.npy')
imgs = [_ for _ in glob.glob(imgDir+'*.*') if _.endswith(fileTypes)]
imgs.sort()
# Extract date and time from filenames
imgdates = [re.search('/20(.+?)_', path).group(1) for path in imgs] # Extract date from filename
imgtimes = [re.search('_(.+?)_', path).group(1) for path in imgs] # Extract time from filename
# Convert to unix datetime 
imgdatetimes = np.array([(datetime.timestamp(datetime(int('20'+imgdates[i][:2]),int(imgdates[i][2:4]),int(imgdates[i][4:6]),int(imgtimes[i][:2])+utcDiff,int(imgtimes[i][2:4]),int(imgtimes[i][4:6])))) for i in range(len(imgs))])*1000

# Imprt paths and get corresponding timestamps for images
pathDf = pd.read_csv(pathFile)
# Get nearest GPS timestamp
gpstimes = [min(pathDf['timestamp'], key=(lambda list_value : abs(list_value - i))) for i in imgdatetimes]

# Create image dataframe
imgDf = pd.DataFrame(data=np.array([imgs,gpstimes]).transpose(),columns=['imgPath','timestamp'])
imgDf['timestamp'] = imgDf['timestamp'].astype(float)

# Merge with path dataframe
merged = imgDf.merge(pathDf[pathColumns], on='timestamp', how='left')
# Filter for acceptable measurement altitudes
merged = merged.loc[(merged[altCol] > minAlt) & (merged[altCol] < maxAlt)]
if imgTimes: merged = merged.loc[(merged['timestamp'] > imgTimes[0]) & (merged['timestamp'] < imgTimes[1])]

# Convert coordinates to UTM
myProj = Proj('+proj=utm +zone='+utmZone+', +'+hemisphere+' +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
merged['x'], merged['y'] = myProj(merged['longitude'].values, merged['latitude'].values)

# Display example
merged.head()

distFilt = 6 # Filter out images where angle is high

fig, axs = plt.subplots(2,2,figsize=[11,8])
# Pitch/roll/yaw plot
axs[0,0].plot(merged.index,merged['pitch(deg)'],label='pitch')
axs[0,0].plot(merged.index,merged['roll(deg)'],label='roll')
par1 = ax.twinx()
par1.plot(merged.index,merged['yaw(deg)'],c='g',label='yaw')
ax.set_title(' Pitch/roll/yaw evolution')
ax.set_xlabel('Image number'), ax.set_ylabel('Degrees (pitch & roll)')
par1.set_ylabel('Degrees (yaw)')
fig.legend(loc='upper left')

# Coordinate corrections
# Angle corrected
dist = merged['altitude(m)']*np.tan(np.deg2rad(merged['pitch(deg)']))
merged['xc'], merged['yc'] = merged.x+(dist*np.sin(np.deg2rad(merged['yaw(deg)']))), merged.y+(dist*np.cos(np.deg2rad(merged['yaw(deg)'])))
if distFilt: idf = merged[abs(dist) < distFilt]
else: idf = merged.copy()
    
## Plot flight path with an example image and downscale
# Plot example image
arr = hf.img_to_arr(merged.iloc[imageNum]['imgPath'])
yCoords, xCoords = [(np.array(range(arr.shape[i]))-(arr.shape[i]/2))*pxSize+pxSize/2 for i in [0,1]]
if useCentre: xq, yq = int(np.floor(len(xCoords)/4)), int(np.floor(len(yCoords)/4))
else: xq, yq = False, False
arr = hf.img_to_arr(merged.iloc[imageNum]['imgPath'], xq=xq, yq=yq)
a = axs[1,0].imshow(arr, cmap=cmap)
axs[1,0].set_title('Image {}'.format(imageNum))

# Plot images to be used
b = axs[0,1].scatter(idf.xc,idf.yc,c=idf.index, cmap='Spectral')
size = np.array(arr.shape[:2])*pxSize/2
for i, row in idf[::skip].iterrows():
    axs[0,1].plot([row.x-size[1], row.x+size[1]], [row.y, row.y],'k-o')
    axs[0,1].plot([row.x, row.x],[row.y-size[1], row.y+size[1]],'k-o')
axs[0,1].set_title('Used image coordinates')
axs[0,1].legend(labels=['Extent for every {}th'.format(skip)])
fig.colorbar(b, ax=axs[0,1],label='Photo number')

# Plot example downsampled image
ds_array = hf.downsample_arr(arr, pxSize, resolution)
c = axs[1,1].imshow(ds_array) if imageType else axs[1,i].imshow(ds_array, vmin=arr.min(), vmax=arr.max(),cmap=cmap)
axs[1,1].set_title('Image {} downsampled to {} m resolution'.format(str(imageNum),resolution))
if imageType is False: fig.colorbar(c, ax=axs[1,1], label=' Temperature ($^{\circ}$C)')
fig.tight_layout()
plt.show()

In [ ]:
# Try xy if you like

In [ ]:
# Multi-image stitching
if 'result' in globals(): del(result)
# Setup variables
start = 1
end = 19
step = 1
skips = [17, 26,29,37,44,45,56]
xmin, xmax, ymin, ymax = 0,0,0,0

# Stitching properties
feature_extractor = 'orb'
feature_matching = 'bf'
tempfiles = ['temp0.jpg','temp1.jpg']
prev = start

arr = hf.img_to_arr(idf.iloc[start]['imgPath'], xq=xq, yq=yq)
single = ((arr+10)*255/40)
imageio.imwrite(tempfiles[0],np.dstack((single,single,single)))
prevImg = np.dstack((single,single,single)).astype(np.uint8)
totalBox=[prevImg.shape[0], prevImg.shape[1]]
prevBox=[0,prevImg.shape[0],0,prevImg.shape[1]]

for imgNum in tqdm(np.arange(start+step if start < end else start-step,end+1,step if start < end else -step)):
    if imgNum in skips:
        continue
    arrs = [hf.img_to_arr(idf.iloc[prev]['imgPath'], xq=xq, yq=yq),hf.img_to_arr(idf.iloc[imgNum]['imgPath'], xq=xq, yq=yq)]

    for i, val in enumerate(tempfiles): 
        single = ((arrs[i]+10)*255/40)
        imageio.imwrite(val,np.dstack((single,single,single)))
    # Read images - Make sure that the train image is the image that will be transformed
    trainImg, queryImg = [imageio.imread(i) for i in tempfiles]
    #for i in tempfiles: os.remove(i)
    # Opencv defines the color channel in the order BGR - transform it to RGB to be compatible to matplotlib
    trainImg_gray, queryImg_gray = [cv2.cvtColor(i, cv2.COLOR_RGB2GRAY) for i in [trainImg, queryImg]]

    # Detect the keypoints and features on both images
    kpsA, featuresA = hf.detectAndDescribe(trainImg_gray, method=feature_extractor)
    kpsB, featuresB = hf.detectAndDescribe(queryImg_gray, method=feature_extractor)

    # Link  the identified features between images
    if feature_matching == 'bf':
        matches = hf.matchKeyPointsBF(featuresA, featuresB, method=feature_extractor)
    elif feature_matching == 'knn':
        matches = hf.matchKeyPointsKNN(featuresA, featuresB, ratio=0.75, method=feature_extractor)

    # Potential to improve by filtering out matches that are not in same direction of travel as drone
    ma = np.array([kpsA[j].pt for j in [i.queryIdx for i in matches]])
    mb = np.array([kpsB[j].pt for j in [i.trainIdx for i in matches]])

    # Filter by only matches in direction of travel
    xmov, ymov = idf.iloc[imgNum]['xc']-idf.iloc[prev]['xc'], idf.iloc[imgNum]['yc']-idf.iloc[prev]['yc']
    bothMov = 2
    if abs(xmov) < bothMov/2 and abs(ymov) < bothMov/2:
        mam, mbm = ma, mb
    else: 
        if abs(xmov) > abs(ymov):
            if xmov < 0:
                mam, mbm = ma[[ma[i,0]>mb[i,0] for i in range(len(ma))]], mb[[ma[i,0]>mb[i,0] for i in range(len(ma))]]
            else: mam, mbm = ma[[ma[i,0]>mb[i,0] for i in range(len(ma))]], mb[[ma[i,0]>mb[i,0] for i in range(len(ma))]]
        else:
            if ymov < 0:
                mam, mbm = ma[[ma[i,1]<mb[i,1] for i in range(len(ma))]], mb[[ma[i,1]<mb[i,1] for i in range(len(ma))]]
            else: mam, mbm = ma[[ma[i,1]>mb[i,1] for i in range(len(ma))]], mb[[ma[i,1]>mb[i,1] for i in range(len(ma))]]
#     xmov, ymov = idf.iloc[imgNum]['yc']-idf.iloc[prev]['yc'], idf.iloc[imgNum]['xc']-idf.iloc[prev]['xc']
#     bothMov = 2
#     if abs(xmov) < bothMov/2 and abs(ymov) < bothMov/2:
#         mam, mbm = ma, mb
#     else: 
#         if abs(xmov) > abs(ymov):
#             if xmov < 0:
#                 #print('hello')
#                 mam, mbm = ma[[ma[i,0]>mb[i,0] for i in range(len(ma))]], mb[[ma[i,0]>mb[i,0] for i in range(len(ma))]]
#             else: 
#                 print('hello')
#                 mam, mbm = ma[[ma[i,0]<mb[i,0] for i in range(len(ma))]], mb[[ma[i,0]<mb[i,0] for i in range(len(ma))]]
#         else:
#             if ymov < 0:
#                 mam, mbm = ma[[ma[i,1]>mb[i,1] for i in range(len(ma))]], mb[[ma[i,1]>mb[i,1] for i in range(len(ma))]]
#             else: mam, mbm = ma[[ma[i,1]<mb[i,1] for i in range(len(ma))]], mb[[ma[i,1]<mb[i,1] for i in range(len(ma))]]

    # Difference between 2 images considered
    diff = np.median(mam-mbm, axis=0).astype(int)
  
    #------------------#    
    if len(mam) > 10 and np.std(mam-mbm, axis=0).mean() < 50:
            # Work out new positions
        newBox=[int(np.round(prevBox[0]+diff[1])), int(np.round(prevBox[1]+diff[1])), int(np.round(prevBox[2]+diff[0])),int(np.round(prevBox[3]+diff[0]))] # New box position before adjustment for expanding total box

        pos = [0,0] # Position for previously merged images
        modBox = [0,0,0,0] # Position for new image
        # If bounds on axis 0 go beyond total
        if newBox[0]<0:
            xmin = imgNum
            totalBox[0]+=abs(newBox[0])
            modBox[1] = newBox[1]-newBox[0]
            modBox[0] = 0
            pos[0] = abs(newBox[0])
        elif newBox[1] > totalBox[0]:
            xmax=imgNum
            totalBox[0]=newBox[1]
            modBox[1] = newBox[1]
            modBox[0] = newBox[0]
            pos[0] = 0
        else: modBox[0], modBox[1], pos[0] = newBox[0], newBox[1], 0
        # If bounds on axis 1 go beyond total
        if newBox[2]<0:
            ymin=imgNum
            totalBox[1]+=abs(newBox[2])
            modBox[3] = newBox[3]-newBox[2]
            modBox[2] = 0
            pos[1] = abs(newBox[2])
        elif newBox[3] > totalBox[1]:
            ymax=imgNum
            totalBox[1]=newBox[3]
            modBox[3] = newBox[3]
            modBox[2] = newBox[2]
            pos[1] = 0
        else: modBox[2], modBox[3], pos[1] = newBox[2], newBox[3], 0    
        prevBox = modBox  
        
        single = (arrs[1]+10)*255/40
        queryImg = np.dstack((single,single,single)).astype(np.uint8)
        #queryImg = [np.dstack((((arrs[i]+10)*255/40),((arrs[i]+10)*255/40),((arrs[i]+10)*255/40))).astype(np.uint8) for i in range(len(imageNums))]
        result = np.zeros([totalBox[0],totalBox[1],3])
        result[pos[0]:pos[0]+prevImg.shape[0],pos[1]:pos[1]+prevImg.shape[1],:] = prevImg
        result[modBox[0]:modBox[1], modBox[2]:modBox[3],:] = queryImg
        print('Images {} and {} merged.'.format(str(prev),str(imgNum)))
        prev=imgNum
        prevImg = result
    else: print('Images {} and {}, poor matching'.format(str(prev),str(imgNum)))
    
    
fig = plt.figure(figsize=(20,10))
#extent = (idf.iloc[imgNum]['xc']-img
plt.imshow(result.mean(axis=2),cmap=cmap)

In [ ]:
# Save as kmz
inputCoords = True
userInputCoords = np.array([[0.106680, 0.108845],[52.204120, 52.204950]])
pixels = 1024 * 10
cmap = colorbrewer.get_map('RdYlBu', 'diverging', 11, reverse=True).mpl_colormap
if inputCoords: conv = userInputCoords
else: conv = np.array(myProj([idf.iloc[xmin]['yc']-pxSize*(np.floor(queryImg.shape[0]/2)+0.5), idf.iloc[xmax]['yc']+pxSize*(np.floor(queryImg.shape[0]/2)+0.5)],
                       [idf.iloc[ymin]['xc']-pxSize*(np.floor(queryImg.shape[1]/2)+0.5), idf.iloc[ymax]['xc']+pxSize*(np.floor(queryImg.shape[1]/2)+0.5)], inverse=True))
fig, ax = hf.gearth_fig(llcrnrlon=conv[0].min(), llcrnrlat=conv[1].min(),
                         urcrnrlon=conv[0].max(), urcrnrlat=conv[1].max(),
                         pixels=pixels)

rot = ndimage.rotate(result.mean(axis=2), 90)
single = np.ma.masked_where(rot < 1e-1, rot)/255*40-10
cs = ax.imshow(single,extent=(conv[0].min(),conv[0].max(),conv[1].min(),conv[1].max()), cmap=cmap, vmin=single.min(),vmax=single.max())
ax.set_axis_off()
fig.savefig('original0-25.png', transparent=True, format='png', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
resolution = 6
fig, ax = hf.gearth_fig(llcrnrlon=conv[0].min(), llcrnrlat=conv[1].min(),
                         urcrnrlon=conv[0].max(), urcrnrlat=conv[1].max(),
                         pixels=pixels)
#ds = hf.downsample_arr(single,pxSize,resolution)
#single = rot/255*50-10
ds = np.ma.masked_where(abs(hf.downsample_arr(single.filled(),pxSize,resolution)) > 1e3, hf.downsample_arr(single,pxSize,resolution))
cs = ax.imshow(ds,extent=(conv[0].min(),conv[0].max(),conv[1].min(),conv[1].max()), cmap=cmap, vmin=single.min(),vmax=single.max())
ax.set_axis_off()
fig.savefig('downsample0-25.png', transparent=True, format='png', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
fig = plt.figure(figsize=(1.0, 4.0))
ax = fig.add_axes([0.07, 0.05, 0.27, 0.9])
cb = fig.colorbar(cs, cax=ax)
cb.set_label('Temperature ($^{\circ}$C)', rotation=-90, color='k', labelpad=20, fontsize=13)
fig.tight_layout()
fig.savefig('legend.png', format='png', bbox_inches = 'tight', pad_inches = 0.2) 

In [ ]:
#Flight Path
stop = 25
x, y = idf.longitude[:stop], idf.latitude[:stop]
fig, ax = hf.gearth_fig(llcrnrlon=conv[0].min(), llcrnrlat=conv[1].min(),
                         urcrnrlon=conv[0].max(), urcrnrlat=conv[1].max(),
                         pixels=pixels)
ax.plot(x,y,'k-',linewidth=2,label='raw')
ax.set_axis_off()
fig.savefig('path0-25.png', transparent=True, format='png', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
hf.make_kml(llcrnrlon=conv[0].min(), llcrnrlat=conv[1].min(),
         urcrnrlon=conv[0].max(), urcrnrlat=conv[1].max(),
         figs=['overlay1.png', 'overlay2.png','path.png'], colorbar='legend.png',
         kmzfile='stitch_thermal.kmz', description=str(resolution)+'m resolution thermal imagery', name='Satellite resolution')

In [ ]:
# Save as .csv file
ds_arr = ds.shape
ptslon = np.linspace(conv[0].min(),conv[0].max(),ds_arr[1]+1)[:-1]
ptslon += (ptslon[1]-ptslon[0])/2
ptslat = np.linspace(conv[1].min(),conv[1].max(),ds_arr[0]+1)[:-1]
ptslat += (ptslat[1]-ptslat[0])/2
lonm, latm = np.meshgrid(ptslon,ptslat)

import pandas as pd
ds_q = int(np.floor(resolution/pxSize))
arr = np.round(single[:-(single.shape[0] % ds_q),:-(single.shape[1] % ds_q)]).astype(int)
most=np.zeros([len(range(0,arr.shape[0]-ds_q,ds_q))+1, len(range(0,arr.shape[1]-ds_q,ds_q))+1])
for i in range(0,arr.shape[0],ds_q):
    for j in range(0,arr.shape[1],ds_q):
        most[int(i/ds_q),int(j/ds_q)] = Counter(arr[i:i+ds_q,j:j+ds_q].reshape(1,-1).tolist()[0]).most_common(1)[0][0]
        
tdf = pd.DataFrame(np.concatenate([np.flip(latm.reshape(-1,1)), lonm.reshape(-1,1), ds.reshape(-1,1), most.reshape(-1,1)], axis=1), columns=['Latitude', 'Longitude', '6m T(C)', '0.27cm T(C)'])
filt = tdf[ds.mask.reshape(-1,1) == False]
filt.to_csv('plots/stitch_temps.csv', index_label='UID')